# Climate change sucks

In [1]:
import csv
import numpy as np

file = open('./combined_dataset.csv')
# file = open('./Annual_Surface_Temperature_Change.csv')
csvreader = csv.reader(file)

countries = []  # To store country names
numerical_data = []  # To store numerical data

next(csvreader, None)

for row in csvreader:
    # Split the data into country name and numerical part
    country_name = row[1]  # First column
    numerical_part = []
    for value in row[6:-2]:
        try:
            numerical_part.append(float(value))
        except ValueError:
            numerical_part.append(None)  

    countries.append(country_name)
    numerical_data.append(numerical_part)

# Convert the data lists to numpy arrays
countries = np.array(countries)
numerical_data = np.array(numerical_data, dtype=float)

print(countries.shape)
print(numerical_data.shape)

(2226,)
(2226, 29)


In [16]:
import csv
import numpy as np

file = open('./combined_dataset.csv')
csvreader = csv.reader(file)

# Skip the header row
next(csvreader, None)

# Initialize a nested dictionary to store data for each country, indicator, and data
country_data = {}

for row in csvreader:
    country_name = row[1]
    indicator = row[4]  # Indicator column
    year_data = row[6:]  # Data from 1992 onwards
    year_data = [float(value) if value != '' else None for value in year_data]

    # Check if the country is already in the dictionary
    if country_name in country_data:
        if indicator in country_data[country_name]:
            country_data[country_name][indicator].append(year_data)
        else:
            country_data[country_name][indicator] = [year_data]
    else:
        country_data[country_name] = {
            indicator: [year_data]
        }

# Convert the lists of data for each country to NumPy arrays
for country, indicators_data in country_data.items():
    for indicator, data in indicators_data.items():
        indicators_data[indicator] = np.array(data, dtype=float)
        
# Example: Accessing data for a specific country, indicator, and year
desired_country = "Afghanistan, Islamic Rep. of"
desired_indicator = "Temperature change"
desired_year_index = 0  # Index for the year you want, e.g., 0 for F1992

if desired_country in country_data:
    country_info = country_data[desired_country]
    if desired_indicator in country_info:
        data_for_indicator = country_info[desired_indicator]
        data_for_year = data_for_indicator[:, desired_year_index]
        print(data_for_year)
    else:
        print(f"Data not found for the indicator: {desired_indicator}")
else:
    print(f"Data not found for {desired_country}")


[-0.294]


In [40]:
# Evaluate the trained models on the validation set
for country in val_countries:
    if any(indicator.startswith("Temperature change") for indicator in country_data[country].keys()):
        # Find the indicator that starts with "Temperature change"
        temperature_change_indicator = next(
            (indicator for indicator in country_data[country].keys() if indicator.startswith("Temperature change")),
            None
        )

        if temperature_change_indicator is not None:
            model = trained_models[country]
            input_data = torch.Tensor(country_data[country][temperature_change_indicator][:, :-2])
            target_data = torch.Tensor(country_data[country][temperature_change_indicator][:, -1])
            with torch.no_grad():
                output = model(input_data)
                loss = criterion(output, target_data)
                evaluation_results[country] = loss.item()

# Example: Print the evaluation results
for country, loss in evaluation_results.items():
    print(f"Validation loss for {country}: {loss}")


KeyError: 'Bhutan'

In [16]:
! pip install torch

                                              0.0/192.3 MB ? eta -:--:--
                                              0.0/192.3 MB ? eta -:--:--
                                              0.0/192.3 MB ? eta -:--:--
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                            0.0/192.3 MB 262.6 kB/s eta 0:12:13
                                             0.0/192.3 MB 81.9 kB/s eta 0:39:08
                                             0.0/192.3 MB 81.9 kB/s eta 0:39:08
                                            0.1/192.3 MB 117.2 kB/s eta 0:27:22
                                            0.1/192.3 MB 148.

Alex working code:

In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

import pennylane as qml

# Define a quantum device
dev = qml.device("default.qubit", wires=1)

# Define a quantum circuit using Pennylane
@qml.qnode(dev)
def quantum_circuit(x):
    qml.RY(x[0], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define a Pennylane hybrid model that combines the quantum circuit and PyTorch model
class HybridModel(nn.Module):
    def __init__(self, input_size):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128).to(torch.float64)
        self.fc2 = nn.Linear(128, 64).to(torch.float64)
        self.fc3 = nn.Linear(64, 1).to(torch.float64)
        self.quantum_layer = quantum_circuit

    def forward(self, x):
        x1 = torch.relu(self.fc1(x))
        x2 = torch.relu(self.fc2(x1))
        output = self.fc3(x2)

        # Pass input through the quantum layer
        quantum_output = self.quantum_layer(x)
        output = output + quantum_output

        return output

# Instantiate the regression model
input_size = 30
model = HybridModel(input_size)

# Define a loss function (Mean Squared Error) and an optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

file = open('./Annual_Surface_Temperature_Change.csv')
csvreader = csv.reader(file)

prev_years = []
final_years = []
first = True
for row in csvreader:
    if row[0] == '225':
        prev_years.append([float(val) if val != "" else 0.0 for val in row[35:len(row)-1]])
        final_years.append([float(row[len(row)-1]) if row[len(row)-1] != "" else 0.0])
        break

    
    prev_years.append([float(val) if val != "" else 0.0 for val in row[35:len(row)-1]])
    final_years.append([float(row[len(row)-1]) if row[len(row)-1] != "" else 0.0])

print(len(prev_years))
file.close()

prev_years = torch.tensor(prev_years, dtype=torch.float64)
final_years = torch.tensor(final_years, dtype=torch.float64)
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(prev_years)
    loss = criterion(output, final_years)
    loss.backward()
    optimizer.step()

# After training, you can use the model to make predictions
model.eval()
cum_error = 0
for i in range(10):
    with torch.no_grad():
        test_input = prev_years[i]
        predicted_output = model(test_input)

    error = abs(predicted_output.item()-final_years[i][0])
    cum_error += error
    print(f"predicted value: {predicted_output.item()}, actual value is: {final_years[i][0]}, error: {error}")
print(f"MSE: {cum_error**2/50}")

200


c:\Users\User\anaconda3\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([200, 1])) that is different to the input size (torch.Size([200, 30])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


predicted value: 2.32373485827063, actual value is: 2.012, error: 0.3117348582706301
predicted value: 2.1468177716427745, actual value is: 1.518, error: 0.6288177716427745
predicted value: 2.0987904131732664, actual value is: 1.688, error: 0.4107904131732665
predicted value: 1.6456199669931562, actual value is: 1.256, error: 0.38961996699315615
predicted value: 1.5471598128631032, actual value is: 1.212, error: 0.33515981286310326
predicted value: 1.2043602960382502, actual value is: 0.839, error: 0.36536029603825027
predicted value: 1.1400627183535335, actual value is: 0.77, error: 0.37006271835353344
predicted value: 0.8321943156891336, actual value is: 0.643, error: 0.18919431568913359
predicted value: 1.4049151722014508, actual value is: 1.707, error: 0.3020848277985493
predicted value: 1.1614612062509562, actual value is: 0.754, error: 0.40746120625095616
MSE: 0.27532447179974645


In [126]:
import torch 
import csv

# Initialize empty lists to store data
prev_years = [[],[]]
final_years = [[],[]]

# Load temperature change data
with open('./Annual_Surface_Temperature_Change.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Assuming the data starts from column 4 and ends at the last column (index -1)
        temperature_data = [float(val) if val else 0.0 for val in row[35:-1]]
        final_temperature = float(row[-1]) if row[-1] else 0.0
        prev_years[0].append(temperature_data)
        final_years[0].append(final_temperature)

# Load total number of disasters data
with open("./disastors.csv", 'r', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter='\t')  # Set delimiter to '\t' for tab-separated values

    for row in csvreader:
        if row[4] == 'Total disasters' and row[5] == 'Number of':
            # Assuming the data starts from column 6 and ends at the last column (index -1)
            disasters_data = [float(val) if val else 0.0 for val in row[6:-1]]
            final_disasters = float(row[-1]) if row[-1] else 0.0
            prev_years[1].append(disasters_data)
            final_years[1].append(final_disasters)

# Convert the lists to PyTorch tensors
prev_years = torch.tensor(prev_years, dtype=torch.float64)
final_years = torch.tensor(final_years, dtype=torch.float64)

training_data = prev_years[:,:150,:]
test_data = prev_years[:,150:,:]

training_labels = final_years[:,:150]
test_labels = final_years[:,150:]

print("training data    ",training_data.shape)
print("test data        ",test_data.shape)
print("training labels  ",training_labels.shape)
print("test labels      ",test_labels.shape)


training data     torch.Size([2, 150, 30])
test data         torch.Size([2, 50, 30])
training labels   torch.Size([2, 150])
test labels       torch.Size([2, 50])


In [128]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import tqdm
import pennylane as qml
import tensorflow as tf

# Define a quantum device
dev = qml.device("default.qubit", wires=1)

# Define a quantum circuit using Pennylane
@qml.qnode(dev)
def quantum_circuit(x):
    qml.RY(x[0], wires=0)
    return qml.expval(qml.PauliZ(0))

# Define a Pennylane hybrid model that combines the quantum circuit and PyTorch model
class HybridModel(nn.Module):
    def __init__(self, input_size):
        super(HybridModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128).to(torch.float64)
        self.fc2 = nn.Linear(128, 64).to(torch.float64)
        self.fc3 = nn.Linear(64, 1).to(torch.float64)
        self.quantum_layer = quantum_circuit

    def forward(self, x):
        x1 = torch.relu(self.fc1(x))
        x2 = torch.relu(self.fc2(x1))
        output = self.fc3(x2)

        # Pass input through the quantum layer
        quantum_output = self.quantum_layer(x)        
        output = output + quantum_output
        
        return output

# Load and preprocess your temperature change and total number of disasters data
# Assuming you have organized the data and loaded it into prev_years and final_years

# Instantiate the regression model
input_size = 30
model = HybridModel(input_size)

# Define a loss function (Mean Squared Error) and an optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for i in range(30):
        model.train()
        optimizer.zero_grad()
        
        output_temp = model(training_data[0])
        loss = criterion(output_temp[:,i], training_labels[0])
        print(epoch, i, loss)
        loss.backward()
        optimizer.step()
        
        output_dist = model(training_data[1])
        loss = criterion(output_dist[:,i], training_labels[1])
        print(epoch, i, loss)
        loss.backward()
        optimizer.step()
    

# After training, you can use the model to make predictions
model.eval()
cum_error = 0
for i in range(50):
    with torch.no_grad():
        test_input = test_data[0,i]
        predicted_output = model(test_input)

    error = abs(predicted_output.item()-test_labels[0,i])
    cum_error += error
    print(f"predicted value: {predicted_output.item()}, actual value is: {test_labels[0,i]}, error: {error}\n")
print(f"MSE: {cum_error**2/50}")


0 0 tensor(0.5458, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 0 tensor(8.6094, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 1 tensor(0.4549, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 1 tensor(9.5254, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 2 tensor(0.4213, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 2 tensor(4.3188, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 3 tensor(0.3915, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 3 tensor(7.1984, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 4 tensor(0.4035, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 4 tensor(3.6330, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 5 tensor(0.3770, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 5 tensor(5.5703, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 6 tensor(0.3505, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 6 tensor(3.9116, dtype=torch.float64, grad_fn=<MseLossBackward0>)
0 7 tensor(0.3102, dtype=torch.float64, grad_fn=